<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/Dynamic_fire_test_urban.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%time 
!apt update
!apt upgrade
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 

# install rasterio
!pip install rasterio

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [83.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/multiv

In [3]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import sys
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from osgeo import gdal_array
import numpy as np
import pandas as pd
import imageio
import random

In [9]:
path = '/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/Colab_folder'
!ls '/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/Colab_folder'

GD_fuel_map_crop.tif  GD_ignit_prob_crop.tif  GD_wind.csv
GD_fuel_map.tif       GD_ignit_prob.tif


In [0]:
def load_data(path_to_data=path):
    # load building map as a "fuel" map
    fuel = gdal_array.LoadFile(os.path.join(path_to_data, 'GD_fuel_map_crop.tif'))  # 0 = no fuel; 1 = fuel
    fuel[fuel < 0] = 0
    # load probability of building ignition as an array
    ignition = gdal_array.LoadFile(os.path.join(path_to_data, 'GD_ignit_prob_crop.tif'))  # probability from 0 to 1
    ignition[ignition < 0] = 0
    # load wind data
    wind = pd.read_csv(os.path.join(path_to_data, 'GD_wind.csv'))
    return fuel, ignition, wind


fuel_map, ignition_probability_map, wind_df = load_data()

In [28]:
# test 
# np.set_printoptions(threshold=20)
# fuel_map.shape
# ignition_probability_map.min()

0.0

In [0]:
def wind_scenario(wind_data=wind_df):
    i = np.random.randint(0, wind_data.shape[0])
    wind = wind_data.iloc[i, 2]
    distance = wind_data.iloc[i, 1]
    return wind, distance

In [142]:
# test to replace fire initiation
random_map = np.random.random((ignition.shape[0], ignition.shape[1]))

ignition_map = ignition_probability_map - random_map
ignition_map[ignition_map < 0] = 0
ignition_map[ignition_map > 0] = 1

zeros = np.zeros((fuel_map.shape[0], fuel_map.shape[1], time_total - 1))
ignition_map = np.expand_dims(ignition_map, axis=2)

ignition = np.dstack((zeros, ignition_map))

ignition = np.moveaxis(ignition, -1, 0)
ignition.shape

(100, 1200, 1200)

In [0]:
def fire_propagation(scenarios, fuel=fuel_map, ignition_map=ignition_probability_map):
    fire_list = []
    for scenario in range(scenarios):
        wind, distance = wind_scenario()
        time_total = 1000
        # initialize fire by creating random ignition from ignition probability map
        random_map = np.random.random((ignition.shape[0], ignition.shape[1]))
        ignition_map = ignition_probability_map - random_map
        ignition_map[ignition_map < 0] = 0
        ignition_map[ignition_map > 0] = 1
        zeros = np.zeros((fuel_map.shape[0], fuel_map.shape[1], time_total - 1))
        ignition_map = np.expand_dims(ignition_map, axis=2)
        ignition = np.dstack((zeros, ignition_map))
        ignition = np.moveaxis(ignition, -1, 0)
        # initial fire map
        fire = np.zeros((time_total, fuel_map.shape[0], fuel_map.shape[1]))
        fire[0] = ignition[0] + fuel  # 0 = no fuel, 1 = fuel, 2 = fire

        for time in range(1, time_total, 1):
            # Make a copy of the original fire
            fire[time] = fire[time - 1].copy()
            for x in range(1, fuel[0] - 1):
                for y in range(1, fuel[1] - 1):
                    for d in range(distance, 1, -1):
                        if fire[time - 1, x, y] == 2 and wind == 'buffer':  # It's on fire
                            # If there's fuel surrounding it
                            # set it on fire!
                            if fire[time - 1, x - distance, y + distance] == 1:
                                fire[time, x - distance, y + distance] = 2
                            if fire[time - 1, x, y + distance] == 1:
                                fire[time, x, y + distance] = 2
                            if fire[time - 1, x + distance, y + distance] == 1:
                                fire[time, x + distance, y + distance] = 2
                            if fire[time - 1, x + distance, y] == 1:
                                fire[time, x + distance, y] = 2
                            if fire[time - 1, x + distance, y - distance] == 1:
                                fire[time, x + distance, y - distance] = 2
                            if fire[time - 1, x, y - distance] == 1:
                                fire[time, x, y - distance] = 2
                            if fire[time - 1, x - distance, y - distance] == 1:
                                fire[time, x - distance, y - distance] = 2
                            if fire[time - 1, x - distance, y] == 1:
                                fire[time, x - distance, y] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'N':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x - distance, y + distance] == 1:
                                fire[time, x - distance, y + distance] = 2
                            if fire[time - 1, x, y + distance] == 1:
                                fire[time, x, y + distance] = 2
                            if fire[time - 1, x + distance, y + distance] == 1:
                                fire[time, x + distance, y + distance] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'NE':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x, y + distance] == 1:
                                fire[time, x, y + distance] = 2
                            if fire[time - 1, x + distance, y + distance] == 1:
                                fire[time, x + distance, y + distance] = 2
                            if fire[time - 1, x + distance, y] == 1:
                                fire[time, x + distance, y] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'E':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x + distance, y + distance] == 1:
                                fire[time, x + distance, y + distance] = 2
                            if fire[time - 1, x + distance, y] == 1:
                                fire[time, x + distance, y] = 2
                            if fire[time - 1, x + distance, y - distance] == 1:
                                fire[time, x + distance, y + distance] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'SE':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x + distance, y] == 1:
                                fire[time, x + distance, y] = 2
                            if fire[time - 1, x + distance, y - distance] == 1:
                                fire[time, x + distance, y + distance] = 2
                            if fire[time - 1, x, y - distance] == 1:
                                fire[time, x, y - distance] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'S':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x, y - distance] == 1:
                                fire[time, x, y - distance] = 2
                            if fire[time - 1, x - distance, y - distance] == 1:
                                fire[time, x - distance, y - distance] = 2
                            if fire[time - 1, x + distance, y - distance] == 1:
                                fire[time, x + distance, y - distance] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'SW':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x, y - distance] == 1:
                                fire[time, x, y - distance] = 2
                            if fire[time - 1, x - distance, y - distance] == 1:
                                fire[time, x - distance, y - distance] = 2
                            if fire[time - 1, x - distance, y] == 1:
                                fire[time, x - distance, y] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'W':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x - distance, y + distance] == 1:
                                fire[time, x - distance, y + distance] = 2
                            if fire[time - 1, x - distance, y - distance] == 1:
                                fire[time, x - distance, y - distance] = 2
                            if fire[time - 1, x - distance, y] == 1:
                                fire[time, x - distance, y] = 2

                        if fire[time - 1, x, y] == 2 and wind == 'NW':  # It's on fire
                            # set it on fire!
                            if fire[time - 1, x - distance, y + distance] == 1:
                                fire[time, x - distance, y + distance] = 2
                            if fire[time - 1, x - distance, y] == 1:
                                fire[time, x - distance, y] = 2
                            if fire[time - 1, x, y + distance] == 1:
                                fire[time, x, y + distance] = 2

                        fire[time].plot()

                        if np.array_equal(fire[time], fire[time - 1]) == True:
                            fire_list.append(fire[time])
                            pass
                        else:
                            continue
        return fire_list

In [35]:
final_fire_maps = fire_propagation(1)

ValueError: ignored

In [0]:
fire_impact = sum(final_fire_maps)